
## Chain of Thought (CoT) Creation

In this notebook, we aim to create a Chain of Thought (CoT) for clinical trial studies using HuggingFace's transformers and other relevant libraries. Below is a step-by-step guide to our workflow:

1. **Setup and Initialization**:
  - We initialize the ChromaDB client and load the SentenceTransformer model for encoding queries.
  - We define a function to fix invalid JSON strings, which is crucial for handling the metadata of clinical trial studies.

2. **Retrieving Relevant Studies**:
  - We define a function `retrieve_relevant_studies` that queries the ChromaDB collection to find studies relevant to a given query, excluding the study already present in the query.

3. **Crafting Context from Studies**:
  - We define a function `craft_context_from_studies_documents` to create a context string from the documents of related studies. This context is used to provide examples in the CoT creation process.

4. **Generating Messages for CoT**:
  - We define a function `get_messages_for_create_CoT` that generates the system and user messages required for creating a CoT. These messages include the study title, description, and desired criteria.

5. **Prompt Creation**:
  - We define a function `get_prompt_from_studies` that uses the above functions to generate the complete prompt for a given study. This prompt includes the context from related studies and the task instructions for generating the CoT.

6. **Model Inference**:
  - We load the HuggingFace model and tokenizer, and define a function `pipe` to generate the CoT using the model. The function takes the generated messages as input and returns the model's output.
  - For Gemini, can use the function in the gemini section to generate the CoT.

By following this workflow, we can systematically generate a Chain of Thought for clinical trial studies, leveraging the capabilities of HuggingFace's transformers and other relevant tools.
```

In [1]:
import chromadb
from sentence_transformers import SentenceTransformer
from client import Client
from prompt_gen import PromptGen

client = chromadb.PersistentClient(path="./clinical_trials_chroma_all")
embed_model = SentenceTransformer("malteos/scincl")
collection = client.get_or_create_collection("clinical_trials_studies")


client = Client(
  client=client,
  collection=collection,
  embed_model=embed_model
)

prompt_gen = PromptGen(
  client=client
)


/home/swiss/miniconda3/envs/ML/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
print(
    client.retrieve_relevant_studies("Effect of Kinesiotaping on Edema Management, Pain and Function on Patients With Bilateral Total Knee Arthroplasty [SEP] After being informed about the study and potential risk, all patients undergoing inpatient rehabilitation after bilateral total knee arthroplasty will have Kinesio(R)Tape applied to one randomly selected leg while the other leg serves as a control. Measurement of bilateral leg circumference, knee range of motion, numerical rating scale for pain, and selected questions from the Knee Injury and Osteoarthritis Outcome Score will occur at regular intervals throughout the rehabilitation stay. Patients will receive standard rehabilitation.", 
                              "NCT05013879"))

[{'id': 'NCT05457686', 'distance': 30.776390075683594, 'document': '{"metadata": {"NCT_ID": "NCT05457686", "Brief_Title": "The Efficacy of Kinesiotaping on Pain, Edema and Functionality After Total Knee Arthroplasty", "Official_title": "The Efficacy of Kinesiology Taping After Total Knee Arthroplasty Surgery", "Conditions": ["Knee Arthroplasty", "Pain", "Edema"], "Interventions": ["Other: Exercise", "Other: Sham taping", "Other: kinesio tape"], "Location_Countries": ["Turkey"], "Study_Design": {"Study_Type": "INTERVENTIONAL", "Phase": ["NA"], "Primary_Purpose": "TREATMENT", "Allocation": "RANDOMIZED", "Interventional_Model": "PARALLEL", "Masking": "DOUBLE"}, "Recruitment_Information": {"Study_Start_Date(Actual)": "2022-07-14", "Primary_Completion_Date(Actual)(Final_data_collection_date_for_primary_outcome_measure)": "2023-02-12", "Study_Completion_Date(Actual)": "2023-02-22"}, "Study_Record_Dates": {"Study_Registration_Dates": {"First_Submitted": "2022-07-03", "First_Posted(Estimated)"

## Using Llama 3.1

In [3]:
# from vllm import LLM, SamplingParams
# from transformers import AutoTokenizer

# model_id = "neuralmagic/Meta-Llama-3.1-8B-Instruct-quantized.w8a16"
# number_gpus = 1

# tokenizer = AutoTokenizer.from_pretrained(model_id)

# llm = LLM(model=model_id, tensor_parallel_size=number_gpus, max_model_len=20000)

# def pipe(messages):
#     sampling_params = SamplingParams(temperature=0, top_p=0.9, max_tokens=4096)
#     prompts = tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)
#     outputs = llm.generate(prompts, sampling_params)
#     return [i.outputs[0].text for i in outputs]

In [4]:
# import pandas as pd
# from datasets import load_dataset
# from tqdm import tqdm

# # Load the JSON data
# ravis_dataset = load_dataset("ravistech/clinical-trial-llm-cancer-restructure")

# # Initialize an empty DataFrame
# df = pd.DataFrame(columns=['encoded_related_studies', 'title', 'description', 'desired_criteria', 'messages'])

# for study in tqdm(ravis_dataset['train']):
#     print(f"Processing {study}")
#     info_for_prompt = prompt_gen.get_info_for_prompt_gen(study)
    
#     if info_for_prompt:
#         encoded_related_studies, title, description, desired_criteria = info_for_prompt
#         messages = prompt_gen.get_messages_for_CoT_huggingface(encoded_related_studies, title, description, desired_criteria)
        
#         # Add the data to the DataFrame
#         df = df.append({
#             'encoded_related_studies': encoded_related_studies,
#             'title': title,
#             'description': description,
#             'desired_criteria': desired_criteria,
#             'messages': messages
#         }, ignore_index=True)
        
#         print(f"Prompt: {messages}")
#         print(f"Response: {pipe(messages)}")

# # Save the DataFrame to a CSV file
# df.to_csv('responses_gemini.csv', index=False)




## Using Gemini

In [ ]:
import time
import pandas as pd
import json
from datasets import load_dataset
from tqdm import tqdm
import vertexai
from vertexai.batch_prediction import BatchPredictionJob
import os
import uuid

PROJECT_ID = "PROJECT-ID"  # update with Google Cloud project ID
BUCKET_NAME = "BUCKET-NAME" # create a bucket in Google Cloud Storage first
OUTPUT_DIR = "./output/"
CSV_OUTPUT_PATH = "./responses_gemini.csv"
vertexai.init(project=PROJECT_ID, location="us-central1")

ravis_dataset = load_dataset("ravistech/clinical-trial-llm-Open_condition_Cleaned_dup_NCT_ID")

batch_prompts = []
info_data = {}  # key is uuid, value is the info for the prompt
for study in tqdm(ravis_dataset['train']):
    info_for_prompt = prompt_gen.get_info_for_prompt_gen(study)
    
    if info_for_prompt:
        unique_id = str(uuid.uuid4())  # gen uuid for each entries
        encoded_related_studies, title, description, desired_criteria = info_for_prompt
        messages = prompt_gen.user_prompt_template(encoded_related_studies, title, description, desired_criteria)

        request_format = {
            "id": unique_id,
            "request": {
                "contents": [
                    {
                        "role": "user",
                        "parts": [
                            {"text": messages}
                        ]
                    }
                ],
                "system_instruction": {
                    "parts": [
                        {"text": prompt_gen.system_prompt}
                    ]
                }
            }
        }
        batch_prompts.append(request_format)
        
        # store info with unique ID for later matching with the response
        info_data[unique_id] = {
            "encoded_related_studies": encoded_related_studies,
            "title": title,
            "description": description,
            "desired_criteria": desired_criteria,
            "messages": messages,
            "response": None
        }

input_jsonl_path = f'gs://{BUCKET_NAME}/prompt_for_batch_gemini_predict.jsonl'
with open('./prompt_for_batch_gemini_predict.jsonl', 'w') as f:
    for prompt in batch_prompts:
        f.write(json.dumps(prompt) + '\n')

# # Upload the JSONL file to GCS
!gsutil cp ./prompt_for_batch_gemini_predict.jsonl {input_jsonl_path}

batch_prediction_job = BatchPredictionJob.submit(
    source_model="gemini-1.5-flash-002",
    input_dataset=input_jsonl_path,
    output_uri_prefix=f'gs://{BUCKET_NAME}/output/',
)

print(f"Job resource name: {batch_prediction_job.resource_name}")
print(f"Model resource name with the job: {batch_prediction_job.model_name}")
print(f"Job state: {batch_prediction_job.state.name}")

# Check the job status
while not batch_prediction_job.has_ended:
    time.sleep(5)
    batch_prediction_job.refresh()

# Process the output if the job is successful
if batch_prediction_job.has_succeeded:
    print("Job succeeded!")
    output_location = batch_prediction_job.output_location + "/predictions.jsonl"
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    local_output_path = os.path.join(OUTPUT_DIR, "predictions.jsonl")

    # Download the output file from GCS
    !gsutil cp {output_location} {local_output_path}
    print(f"Output file downloaded to: {local_output_path}")
    
    with open(local_output_path, 'r') as f:
        for line in f:
            response_data = json.loads(line)
            unique_id = response_data.get("id")
            response = response_data.get("response", {})
            
            if unique_id in info_data:
                info_data[unique_id]["response"] = response

    df = pd.DataFrame.from_dict(info_data, orient="index")
    df.to_csv(CSV_OUTPUT_PATH, index=False)
    print(f"Responses saved to CSV at: {CSV_OUTPUT_PATH}")
else:
    print(f"Job failed: {batch_prediction_job.error}")

print(f"Job output location: {batch_prediction_job.output_location}")



 12%|█▏        | 26409/213877 [04:52<34:08, 91.50it/s]  